# Find Unique
 - Deliverables:
     - findUnique.py - 50 points ( We will test this code)
     - Notebook with Inspection materials and Results ( 5 pts )
     - required input from STDIN
     - required output to STDOUT
     - output must be sorted by tRNA header

## Sets and tRNA

Mitochondrial tRNA are encoded in the mitochondrial genome and account for ~10% of the coding space, yet over 50% of mitochondrial genomic disease have their origin in this molecule. These molecules are transcribed, processed, modified, and ultimately folded to produce these mature adapters between the mitochondrial transcription and translation processes. Defects in mt.tRNA maturation reflect additional disease states .. if only we could identify those mutations. We are working on such a device, though we first need to use it to identify abundance of these molecules in a mixed population. Are there any unique subsequences among the 22 mt.tRNA that can be used as "tags"? If so, we can count those tags to assess abundance.

## Assignment

Write a python command line program that reads a file of fasta sequences from STDIN, finds the unique subsequences that occur in each single tRNA such that no members of this set occur among any of the other tRNA sets. Each of those 22 sets should be minimized, such that no member of that unique subsequence set is a substring of any other member of this set. [ Unique and Essential]

As an example, let's say that both ACG and AAACGA are in a unique set. Since ACG is a substring of AAACGA we would remove AAACGA. [ ACG is Essential ]

Use Python sets for this assignment[__required__]. Not only will your code be smaller, but it will be more likely to work. The union, intersection and difference operators will be quite useful.

## Rough design plan...

1) compute the set of all substrings from each tRNA sequence. [powerset] I will refer to a set of substrings as a tRNAset.

2) for each tRNAset, compute the union of all other tRNA sets, and then remove that union from the current tRNAset. Notice that this union operation finds all of the substrings from all other tRNA. If any of those are present in your current tRNA, then they are not unique ! [ Unique]

3) for each unique tRNAset, it now contains the truly unique ones, along with any extensions of that subsequence. If, for example, it was found that G only occurred in a single tRNA, then adding an A onto that G must also be unique because it has a G in it. We only want the minimal form.. G. [ Essential]

4) Remove spaces from the header line before sorting and printing. This will make your output a little prettier.

5) make sure to remove any alignment characters from your initial sequences. These characters are periods(.), underscores(\_), or dashes(-) . You will find many new characters in the sequence other than {ACGU} - leave these in place, they are modified bases.

## Report
Print a report that contains items as follows. 

 - Line 1: the tRNA name
 - Line 2: the tRNA sequence ( with the alignment characters removed)
 - lines 3-80 or so, each unique element.

These unique elements need to be ordered by their starting position in the tRNA sequence, and should be spaced over so they are directly under that subsequence in the original tRNA. This looks like an alignment, but you can find where it belongs by using the string.find() method. Include dots in all positions to the left, serving as alignment guides for your reader. [ see sample output below ]

Do this for all of the 22 tRNA sequences.

Print the tRNA out as above, sorted by the header line.  

## Hints:

__use sets and the set operators - this is required !__

your final code will be under 100 lines.

Do most of your coding using methods defined in a class and then instantiated as objects. 

The sequences include characters that are just alignment characters. They are not part of the sequence and must be removed. [-\_\.] are alignment characters. 

When removing items from a tRNAset, don't do this while iterating through that set. Also, when building a unique set you will need the original contents of all other tRNAsets. So.. build a new set to keep the unique contents, or keep track of the elements you intend to delete later. Notice that you build the union of all other tRNA, and this happens 22 times - these unions are all distinct from each other. Example, consider 4 sets, A,B,C,D.  we would compute the set B, C, D to use against set A, and we would compute the set A, C, D to use against B. A and B need to not change while we are doing this computation.

There are cases where a unique element is present multiple times in a single tRNA. This sequence is unique to this tRNA and should be reported in its multiple locations ( same sequence on multiple lines of the output).

This is a command line program, though it does not have any optional parameters. You really don't need the commandline class. Your input comes from STDIN and output goes to STDOUT. Use the FastaReader for input and print statement for output. If you are going to use jupyter to develop your code, you can add a filename to fastareader for your testing.

## Submission

Submit code using Canvas. As always, work with your group and write your own code.

Inspection Intro:
My group worked together to develop the program design, then helped eachother work through bugs in our individual code. I chose to create a separate method for generating the powersets, unique sets, and essential sets. I also decided to generate the desired output in main() rather than in a class method. I handled multiple instances of an essential for one tRNA by using a while loop that iterates through the sequence, appending positions of an essential until it can't find anymore. I now get the desired output, save for an issue with windows that changes one of the special characters in the input file to something else. 

Inspection Conclusion:
This lab felt a lot easier than Lab5 and took me about half of the time to complete. I originally had an issue with my powerset method that caused my output to be misisng an essential or two and to misrepresent some essential by one or two nucleotides. Dr. Bernick helped me find teh bug that was causing this, and I fixed it by changing the indeces I use to take slices of teh sequence in my powerset method. 

In [5]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
# Name: Madison Price (maeprice)
# Group Members: Kiana Imani (kimani), Queenie Li (qli86), and Ellizabeth Beer (ebeer)

import sys
class FastAreader :
    '''
    Define objects to read FastA files.

    instantiation:
    thisReader = FastAreader ('testTiny.fa')
    usage:
    for head, seq in thisReader.readFasta():
        print (head,seq)
    '''
    
    def __init__(self, fname=''):
        '''contructor: saves attribute fname '''
        self.fname = fname
            
    def doOpen(self):
        ''' Handle file opens, allowing STDIN.'''
        if self.fname == '':
            return sys.stdin
        else:
            return open(self.fname)
        
    def readFasta(self):
        ''' Read an entire FastA record and return the sequence header/sequence.'''
        header = ''
        sequence = ''

        with self.doOpen() as fileH:
            
            header = ''
            sequence = ''
            
            # skip to first fasta header
            line = fileH.readline()
            while not line.startswith('>') :
                if not line: # we are at EOF
                    return header, sequence
                line = fileH.readline()
            header = line[1:].rstrip()

            for line in fileH:
                if line.startswith ('>'):
                    yield header,sequence
                    header = line[1:].rstrip()
                    sequence = ''
                else :
                    sequence += ''.join(line.rstrip().split()).upper()

        yield header,sequence

########################################################################

class TRNA:
    '''Analyze a portion of a fastA file containing a tRNA sequence and generate its essential subsequences.
    
    Input: tRNA header and sequence
    Output: 'Essential' substrings, formatted to be aligned with where they occur in the sequence.
    
    '''
    
    tRNAlist = []
    
    def __init__(self,head,seq):
        '''Initialize a TRNA object with all attributes given at instantiation time.'''
        self.head = head.replace(' ','')                      # get rid of white space in sequence 
        self.seq = seq.replace('.','').replace('_','').replace('-','')   # get rid of undesired characters in sequence
        self.powerset = self.getPowerset()         
        self.uniques = set()
        self.essentials = set()
        self.nonessentials = set()
        
    def getPowerset(self):
        '''Generate the powerset of a given tRNA sequence.'''
        powerset = set()
        for nucleotide in range(len(self.seq)):                  # for each nucleotide, generate all possible subsequences starting at that nucleotide 
            for substringEnd in range(nucleotide+1, len(self.seq)+1):  
                powerset.add(self.seq[nucleotide:substringEnd])
        return powerset
    
    def getUniques(self):
        '''Generate the unique subsequences of a tRNA sequence.'''
        others = set()                                                   # initialize accumulating union of 'other' powersets
        for tRNA in TRNA.tRNAlist:
            if tRNA != self:                                             # if tRNA = 'other'
                others = others.union(tRNA.powerset)                     # add this 'other's powerset to accumulating union
        self.uniques = self.powerset - others
        return self.uniques
    
    def getEssentials(self):
        '''Find the shortest unique subsequences aka 'essetials' in a tRNA sequence. '''  
        for unique in self.uniques:                                   
            if unique[:-1] in self.uniques:                              # test if shortening sequence by one nuc yields a subsequnce that is still unique 
                self.nonessentials.add(unique)                           # if still unique, sequence is nonessential
            elif unique[1:] in self.uniques:
                self.nonessentials.add(unique)
            else:
                pass
        self.essentials = self.uniques - self.nonessentials              # generate uniques by subtracting the nonessential set from the unique set
        return self.essentials

#######################################################################
# Main
# Here is the main program
# 
########################################################################
#from sequenceAnalysis import FastAreader

'''
This program uses the FastAreader class and TRNA class to read tRNA headers and sequences from a FastA file and 
output all of the subsequences that are 'essential', or the smallest unique subsequences. 

Input: FastA reader 
Output: Formatted essential subsequences
'''


def main(inCL=None):
    '''For every tRNA header and sequence in a FastA file, generate the essential substrings and output them in 
    the desired format.
    
    Input: FastA file
    Output: Essential substrings for each tRNA sequence
    '''
    
    myFile = FastAreader()
    
    for head,seq in myFile.readFasta():                               # iterate through tRNA headers and sequences
        myTRNA = TRNA(head,seq)                                       # instantiate a TRNA class object for every header, sequence
        TRNA.tRNAlist.append(myTRNA)                                  
    
    TRNA.tRNAlist.sort(key = lambda x : x.head)                       # alphabetize tRNA names
    
    for tRNA in TRNA.tRNAlist:                                        # iterate through sorted tRNA list 
        print(tRNA.head)
        print(tRNA.seq)
        tRNA.getPowerset()
        tRNA.getUniques()
        essentials = tRNA.getEssentials()
        
        essentialPositions = []                                       # instantiate a list to store essentials and their positions
        for essential in essentials:
            position = 0
            while position < len(tRNA.seq):                           # iterate through tRNA sequence
                try:
                    position = tRNA.seq.index(essential, position)    # find each occurence of a essential, changing starting position each time you find one 
                    essentialPositions.append((position, essential))
                except ValueError:
                    break
                position += 1
            #position = tRNA.seq.find(essential)
            #essentialPositions.append((position,essential))
        sortedEssentials = sorted(essentialPositions, key=lambda x: x[0])   # sort list of tuples based on the first field in each tuple (positions)
        
        for essential in sortedEssentials:        # print the appropriate number of periods for each essential, followed by the esential itself 
            print('.'*essential[0], end='')
            print(essential[1])
    
        
if __name__ == "__main__":
    main()  


tRNA|Ala|UGC|Bostaurus|mitochondrial
GAGGAUUU"LCUUAAUUAAAGULGPUGAUUUGCAUPCAAUUGAUGUAAGGUGPAGUCUUGCAAUCCUUACCA
GAGGA
..GGAU
...GAUUU"
......UU"LC
.........LCUUAAU
...........UUAAUUA
..............AUUAAA
...............UUAAAG
...................AGUL
.....................ULG
......................LGP
.......................GPUGA
........................PUGAU
...........................AUUUG
............................UUUGC
..............................UGCAU
.................................AUPC
...................................PCAA
....................................CAAUUG
.....................................AAUUGA
......................................AUUGAUG
.........................................GAUGUA
...........................................UGUAAGG
...............................................AGGUG
...................................................GPA
....................................................PAGU
.....................................................AGUCUU
....

....................UACAG
.....................ACAGC
........................GCUG
.........................CUGACU
..........................UGACUU
............................ACUUCC
..............................UUCCAA
.................................CAAP
..................................AAPCA
...................................APCAG
....................................PCAGCUA
.....................................CAGCUAG
........................................CUAGU
.........................................UAGUUU
...........................................GUUUCG
..............................................UCGG
...............................................CGGU
................................................GGUCU
...................................................CUAGU
....................................................UAGUC
.......................................................UCCG
.........................................................CGAAAAA
................................

CACUA
.ACUAAG
..CUAAGA
.....AGA"
.......A"LCU
.........LCUA
..........CUAUAU
...........UAUAUA
............AUAUAG
..............AUAGC
...............UAGCA
..................CACP
...................ACPA
.....................PAAC
......................AACCU
.........................CU7
...........................7UU
..............................6AG
................................GUUAGA
.................................UUAGAG
..................................UAGAGA
....................................GAGAUUG
........................................UUGAG
.........................................UGAGAG
..........................................GAGAGC
............................................GAGCCA
..............................................GCCAU
................................................CAU"
..................................................U"U
..................................................."UA
....................................................UACUC
....................

# Sample Output

In [ ]:
tRNA|Lys|∃UU|Bostaurus|mitochondrial
CACUAAGA"LCUAUAUAGCACPAACCU∃UU6AGUUAGAGAUUGAGAGCCAU"UACUCUCCUUGGUGACCA
CACU
.ACUA
...UAA
....AAG
.......A"
.........L
..........CUAU
............AUAU
..............AUAG
...............UAGC
.................GCA
.....................P
......................AAC
.......................ACCU
...........................∃
..............................6
...............................AGU
................................GUU
.................................UUA
..................................UAGA
...................................AGAGA
......................................GAU
.......................................AUU
........................................UUGA
.........................................UGAG
..........................................GAGAG
............................................GAGC
..............................................GCC
................................................CAU
..................................................U"
..................................................."U
....................................................UAC
.....................................................ACUC
.......................................................UCU
.........................................................UCC
...........................................................CUU
..............................................................GG
...............................................................GUG
.................................................................GAC
..................................................................ACCA